In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from tqdm import tqdm
import seaborn as sns

# 设置随机种子
def set_seed():
    np.random.seed(100)

# 使用第一份代码的GBM模拟方法
def simulate_GBM(T, n_steps, X0, sigma):
    dt = T / n_steps
    t = np.linspace(0, T, n_steps)
    dW = np.random.normal(0, np.sqrt(dt), n_steps)
    log_X = np.log(X0) - 0.5 * sigma**2 * t + sigma * np.cumsum(dW)
    X = np.exp(log_X)
    return t, X

# 修改后的不确定性区域价格生成（基于第一份代码）
def uncertainty_zone_price(x, alpha, eta, L, type):
    p = np.zeros(len(x))
    p_discrete = [x[0]]
    tau = [0]
    p[0] = np.round(x[0]/alpha)*alpha
    L = 1
    for i in range(1, len(x)):
        if type == "jumpmore":
            current_vol = np.std(x[max(0,i-100):i])  # 使用最近10步的波动率
            if current_vol < 0.005:
                L_probs = [0.8, 0.2, 0.0]  # 低波动率时小跳跃
            else:
                L_probs = [0.4, 0.5, 0.1]  # 高波动率时大跳跃
            L = np.random.choice([1, 2, 3], p=L_probs)
        else:
            L = 1
        lower_bound = p[i-1] - alpha*(L - 0.5 + eta)
        upper_bound = p[i-1] + alpha*(L - 0.5 + eta)
        
        if x[i] > upper_bound:
            p[i] = p[i-1] + L*alpha
            p_discrete.append(p[i])
            tau.append(i)
        elif x[i] < lower_bound:
            p[i] = p[i-1] - L*alpha
            p_discrete.append(p[i])
            tau.append(i)
        else:
            p[i] = p[i-1]
    
    return p, tau

# Black-Scholes期权定价
def BSCall(S0, K, sigma, T):
    if T <= 0:
        return max(S0 - K, 0)
    sigT = sigma*np.sqrt(T)
    k = np.log(K/S0)
    dp = -k/sigT + 0.5*sigT
    dm = dp - sigT
    return S0*(norm.cdf(dp) - np.exp(k)*norm.cdf(dm))

def delta(S, K, sigma, T):
    if T <= 0:
        return 1.0 if S > K else 0.0
    d1 = (np.log(S/K) + 0.5*sigma**2*T)/(sigma*np.sqrt(T))
    return norm.cdf(d1)

# 计算对冲误差的核心函数
def calculate_hedging_error(t, X, P, tau, K, sigma, use_efficient_price=False):
    portfolio = np.zeros(len(tau))
    positions = np.zeros(len(tau))
    T_total = t[-1]
    
    # 初始头寸
    remaining_T = T_total - t[tau[0]]
    portfolio[0] = BSCall(X[tau[0]], K, sigma, remaining_T)
    positions[0] = delta(X[tau[0]], K, sigma, remaining_T)
    
    for j in range(1, len(tau)):
        # 价格变化计算
        if use_efficient_price:
            price_change = X[tau[j]] - X[tau[j-1]]
        else:
            price_change = P[tau[j]] - P[tau[j-1]]
        
        # 组合价值更新
        portfolio[j] = portfolio[j-1] + positions[j-1] * price_change
        
        # 更新头寸
        remaining_T = T_total - t[tau[j]]
        positions[j] = delta(X[tau[j]], K, sigma, remaining_T)
    
    # 最终支付计算
    final_payoff = max(X[-1] - K, 0)
    return final_payoff - portfolio[-1]

# 参数设置
alpha = 0.05      # Tick size
eta = 0.05         # 不确定性区域参数
L = 1             # 价格跳跃单位
sigma = 0.01      # 波动率
X0 = 100.0        # 初始价格
K = 100           # 行权价
T = 1.0           # 时间长度
n_steps = 100  #time step
number_of_paths = 100 # 蒙特卡洛路径数

# 结果存储
errors_continuous = np.zeros(number_of_paths)  # 连续对冲
errors_discrete = np.zeros(number_of_paths)    # 离散对冲（有效价格）
errors_observed = np.zeros(number_of_paths)    # 离散对冲（观测价格）
errors_final = np.zeros(number_of_paths)    # 离散对冲（策略优化）

# 主循环
set_seed()
for i in tqdm(range(number_of_paths), desc="Processing Paths"):
    # 生成价格路径
    t, X = simulate_GBM(T, n_steps, X0, sigma)
    
    # 生成观测价格和交易时间
    P, tau = uncertainty_zone_price(X, alpha, eta, L,type="jumpmore")
    P_once, tau_once = uncertainty_zone_price(X, alpha, eta, L,type="jumponce")
    
    # 三种对冲策略
    # 1. 连续对冲（无摩擦市场）
    error_cont = calculate_hedging_error(t, X, X, np.arange(n_steps), K, sigma, True)
    
    # 2. 仅在交易时间对冲（使用有效价格）
    error_disc = calculate_hedging_error(t, X, X, tau, K, sigma, True)
    
    # 3. 仅在交易时间对冲（使用观测价格）
    error_obs = calculate_hedging_error(t, X, P, tau, K, sigma, False)

    # 策略4：优化阈值策略
    tau_optimal = []
    current_price = P_once[0]
    for j in range(len(P_once)):
        if abs(P_once[j] - current_price) >= alpha**0.5:#optimal l*alpha
            tau_optimal.append(j)
            current_price = P_once[j]
    errors_final[i] = calculate_hedging_error(t, X, P_once, tau_optimal, K, sigma, False)
    
    errors_continuous[i] = error_cont
    errors_discrete[i] = error_disc
    errors_observed[i] = error_obs

# 可视化结果
plt.figure(figsize=(12, 6))
sns.histplot(errors_continuous, label="Continuous Hedging", bins=20, color='green', alpha=0.7)
#sns.histplot(errors_discrete, label="Discrete (Efficient Price)", bins=20, color='blue', alpha=0.5)
sns.histplot(errors_observed, label="Discrete (Observed Price)", bins=20, color='red', alpha=0.7)
sns.histplot(errors_final, label="Optimal Discrete (Observed Price)", bins=20, color='grey', alpha=0.7)

plt.xlabel("Hedging Error")
plt.ylabel("Frequency")
plt.title("Comparison of Hedging Strategies")
plt.legend()
plt.show()